## Import Dependencies

In [1]:
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, extract
from sqlalchemy import Column, Integer, Float, Date, String, VARCHAR, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

In [2]:
engine = create_engine("sqlite:///../database/db.sqlite")
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
Base.classes.keys()

['athlete_events', 'noc']

In [3]:
# athlete_events = Base.classes.athlete_events
# noc_regions = Base.Classes.noc_regions

session = Session(engine)
inspector = inspect(engine)
inspector.get_table_names()

['athlete_events', 'noc']

In [4]:
athlete_events = Base.classes.athlete_events
noc_regions = Base.classes.noc

columns = inspector.get_columns('athlete_events')
for column in columns:
    print(column["name"], column["type"])

index VARCHAR
ID VARCHAR
Name VARCHAR
Sex VARCHAR
Age INTEGER
Height INTEGER
Weight INTEGER
Team VARCHAR
NOC VARCHAR
Games VARCHAR
Year VARCHAR
Season VARCHAR
City VARCHAR
Sport VARCHAR
Event VARCHAR
Medal VARCHAR


In [5]:
columns = inspector.get_columns('noc')
for column in columns:
    print(column["name"], column["type"])

index VARCHAR
NOC VARCHAR
region VARCHAR
notes VARCHAR


In [6]:
# {
#    "NOC": "USA",
#    "Country": "United States",
#    "Season": "Winter",
#    "Event": "Swimming",
#    "Medals": 120,
#    "Played": 876
#  },

sel = [
    athlete_events.NOC,
    noc_regions.region,
    athlete_events.Season,
    athlete_events.Event,
    athlete_events.Medal
   ]

noc_substring = "AFG"

query = session.query(*sel)\
        .filter(athlete_events.NOC.contains(noc_substring))\
        .filter(athlete_events.NOC == noc_regions.NOC)\
        .all()

df = pd.DataFrame(query)

df = df.groupby(['Season'])

table = df["Event"].value_counts().unstack(fill_value=0)
table['total'] = table.iloc[:,0:].sum(1)
table = table.transpose().reset_index()
table.head()

# merge = pd.merge(df, table, on='Event', how='inner')
# merge.head()

# join = test.join(table, on='Event', how='outer')

Season,Event,Summer
0,Athletics Men's 100 metres,7
1,Athletics Men's 110 metres Hurdles,1
2,Athletics Men's 200 metres,1
3,Athletics Men's 4 x 100 metres Relay,4
4,Athletics Men's 400 metres,1


In [7]:
sel = [
    noc_regions.region,
    athlete_events.Medal
   ]

query = session.query(*sel)\
        .filter(athlete_events.NOC == noc_regions.NOC)\
        .filter(athlete_events.Medal.isnot(None))\
        .all()

df = pd.DataFrame(query)

df = df.groupby(['region'])

table = df["Medal"].count().reset_index()
Total = table['Medal'].sum()

table['measure'] = round((table['Medal'] / Total), 10)

name_dict = {
    'region': 'country'
}

column_list = [
    'country',
    'measure'
]

table = table.rename(columns=name_dict)
table = table[table.columns.intersection(column_list)]

table = table.sort_values(by=['measure'], ascending=False)


df.head()
table.head()


,country,measure
125,USA,0.141726
100,Russia,0.099236
41,Germany,0.094434
124,UK,0.051994
38,France,0.044677


In [8]:
# create two data frames and the conncat them together


sel = [
    noc_regions.region,
    athlete_events.Sport,
    athlete_events.Medal
   ]

query = session.query(*sel)\
        .filter(athlete_events.NOC == noc_regions.NOC)\
        .filter(athlete_events.Medal.isnot(None))\
        .all()

df = pd.DataFrame(query)

group = df.groupby(['region', 'Sport'])
table = group["Medal"].count().reset_index()

no_group = df.groupby(['Sport'])
table_2 =  no_group['Medal'].count().reset_index()
table_2['country'] = 'All'

name_dict = {
    'region': 'country',
    'Medal': 'measure',
    'Sport': 'sports'
}

column_list = [
    'country',
    'sports',
    'measure'
]

table = table.rename(columns=name_dict)
table = table[table.columns.intersection(column_list)]
table = table.sort_values(by=['measure'], ascending=False).reset_index(drop=True)

table_2 = table_2.rename(columns=name_dict)
table_2 = table_2[table_2.columns.intersection(column_list)]
table_2 = table_2.sort_values(by=['measure'], ascending=False).reset_index(drop=True)

result = pd.concat([table_2, table], ignore_index=True)



df.head()
table.head()
table_2.head()
result.head()

C:\Users\Jorge\Anaconda3\envs\ut_data\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,country,measure,sports
0,All,3969,Athletics
1,All,3047,Swimming
2,All,2945,Rowing
3,All,2256,Gymnastics
4,All,1743,Fencing


In [9]:
"""
{
   "total No of athelete": 15,
   "Year": 2000,
   "Season": "winter",
   "Sports": "Basque Pelota"
 },
 {
   "total No of athelete": 15,
   "Year": 2000,
   "Season": "winter",
   "Sports": "Aeronautics"
 },
"""
sel = [
    noc_regions.region,
    athlete_events.Year,
    athlete_events.Season,
    athlete_events.Sport,
    athlete_events.Medal
   ]

query = session.query(*sel)\
        .filter(athlete_events.NOC == noc_regions.NOC)\
        .all()

df = pd.DataFrame(query)
group = df.groupby(['Year', 'Season', 'Sport'])

table = group["Medal"].count().reset_index()

name_dict = {
    'Medal': 'total No of athelete',
    'Sport': 'Sports'
}

column_list = [
    'Year',
    'Season',
    'Sports',
    'total No of athelete'
]

table = table.rename(columns=name_dict)
table = table[table.columns.intersection(column_list)]
# table = table.sort_values(by=['measure'], ascending=False).reset_index(drop=True)

# df.head()
table.head()

,Year,Season,Sports,total No of athelete
0,1896,Summer,Athletics,37
1,1896,Summer,Cycling,16
2,1896,Summer,Fencing,9
3,1896,Summer,Gymnastics,37
4,1896,Summer,Shooting,15


In [20]:
"""
{
    "Country": "CHINA",
    "freq":
    {
        "Gold": "4786",
        "Silver": "1319",
        "Bronze": "249"
    }
},
{
    "Country": "IND",
    "freq":
    {
        "Gold": "1101",
        "Silver": "412",
        "Bronze": "674"
    }
},
"""
sel = [
    athlete_events.NOC,
    athlete_events.Medal
   ]

query = session.query(*sel)\
        .filter(athlete_events.Medal.isnot(None))\
        .all()

df = pd.DataFrame(query)
group = df.groupby(['NOC'])

table = group["Medal"].value_counts().unstack(fill_value=0)
table = table.sort_values('Gold', ascending=False).nlargest(10, 'Gold')
table = table[['Gold', 'Silver', 'Bronze']]

# data = table.to_json(orient='records')
# data = {}

# dict_a = table.to_dict(orient='records')
# data = table.to_json(orient='table')
data = []

dict_a = table.T.to_dict()
for country, values in dict_a.items():
#     print(country)
    obj = {}
    obj['country'] = country
    obj['freq'] = {'Gold':dict_a[country]['Gold'],'Silver':dict_a[country]['Silver'], 'Bronze':dict_a[country]['Bronze'] }
#     print(obj)
    data.append(obj)
data

# print(dict_a)
# print(jsonify(data))
table.head(20)

Medal,Gold,Silver,Bronze
NOC,,,
USA,2638,1641,1358
URS,1082,732,689
GER,745,674,746
GBR,678,739,651
ITA,575,531,531
FRA,501,610,666
SWE,479,522,535
CAN,463,438,451
HUN,432,332,371


In [15]:
"""
{ 
    "country": “All”, 
    "year": "2008",
    "measure": "289309" },
"""
sel = [
    noc_regions.region,
    athlete_events.Year,
    athlete_events.Medal
   ]

query = session.query(*sel)\
        .filter(athlete_events.NOC == noc_regions.NOC)\
        .filter(athlete_events.Season == 'Summer')\
        .filter(athlete_events.Medal.isnot(None))\
        .all()

df = pd.DataFrame(query)

group = df.groupby(['region', 'Year'])
table = group["Medal"].count().reset_index()

no_group = df.groupby(['Year'])
table_2 =  no_group['Medal'].count().reset_index()
table_2['country'] = 'All'

name_dict = {
    'region': 'country',
    'Medal': 'measure',
    'Year': 'year'
}

column_list = [
    'country',
    'year',
    'measure'
]

table = table.rename(columns=name_dict)
table = table[table.columns.intersection(column_list)]
table = table.sort_values(by=['country', 'year'], ascending=False).reset_index(drop=True)

table_2 = table_2.rename(columns=name_dict)
table_2 = table_2[table_2.columns.intersection(column_list)]
table_2 = table_2.sort_values(by=['year'], ascending=True).reset_index(drop=True)

result = pd.concat([table_2, table], ignore_index=True)

result.head()

C:\Users\Jorge\Anaconda3\envs\ut_data\lib\site-packages\ipykernel_launcher.py:48: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,country,measure,year
0,All,143,1896
1,All,604,1900
2,All,486,1904
3,All,458,1906
4,All,831,1908
